# Install Libraries

In [ ]:
!pip install evaluate
!pip install jiwer
!pip install tqdm

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 18.3 MB/s eta 0:00:00


In [ ]:
!ps -ef | grep train_lora_script


root        1338    1204  0 10:35 ?        00:00:00 /bin/bash -c ps -ef | grep train_lora_script
root        1340    1338  0 10:35 ?        00:00:00 grep train_lora_script


# Mount Google Drive

In [ ]:
from google.colab import drive

drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!ls -R /content/drive/MyDrive/project/whisper_lora_results


/content/drive/MyDrive/project/whisper_lora_results:
checkpoint-125	checkpoint-2201  checkpoint-4402  checkpoint-6603

/content/drive/MyDrive/project/whisper_lora_results/checkpoint-125:
adapter_config.json	   preprocessor_config.json  tokenizer_config.json
adapter_model.safetensors  README.md		     tokenizer.json
added_tokens.json	   rng_state.pth	     trainer_state.json
merges.txt		   scaler.pt		     training_args.bin
normalizer.json		   scheduler.pt		     vocab.json
optimizer.pt		   special_tokens_map.json

/content/drive/MyDrive/project/whisper_lora_results/checkpoint-2201:
adapter_config.json	   preprocessor_config.json  tokenizer_config.json
adapter_model.safetensors  README.md		     tokenizer.json
added_tokens.json	   rng_state.pth	     trainer_state.json
merges.txt		   scaler.pt		     training_args.bin
normalizer.json		   scheduler.pt		     vocab.json
optimizer.pt		   special_tokens_map.json

/content/drive/MyDrive/project/whisper_lora_results/checkpoint-4402:
adapter_config.js

# Fine Tune LoRA Model

In [ ]:
# Fine Tune LoRA Model

In [ ]:
import os
import json
from pathlib import Path
import torch
from torch.utils.data import Dataset, DataLoader
import torchaudio
from dataclasses import dataclass
from typing import Any

from transformers import (
    WhisperFeatureExtractor,
    WhisperTokenizer,
    WhisperProcessor,
    WhisperForConditionalGeneration,
    Seq2SeqTrainer,
    Seq2SeqTrainingArguments,
)
from peft import (
    prepare_model_for_kbit_training,
    LoraConfig,
    get_peft_model,
)
import evaluate

# ============================================================
# 1. Configuration
# ============================================================
#DATA_ROOT = Path(r"/content/drive/MyDrive/cadenza_data")
DATA_ROOT = Path("/content/drive/MyDrive")
TRAIN_AUDIO_DIR = DATA_ROOT / "train" / "signals"
VALID_AUDIO_DIR = DATA_ROOT / "valid" / "signals"
TRAIN_METADATA_PATH = DATA_ROOT / "metadata" / "train_metadata.json"
VALID_METADATA_PATH = DATA_ROOT / "metadata" / "valid_metadata.json"  # optional

MODEL_NAME = "openai/whisper-small"  # or "openai/whisper-large-v2"
LANGUAGE = "English"
TASK = "transcribe"

EXPERIMENT_ID = "exp4"
EXPERIMENT_NAME = f"whisper_lora_{EXPERIMENT_ID}"

OUTPUT_DIR = f"/content/drive/MyDrive/project/{EXPERIMENT_NAME}_results"
SAVE_DIR = f"/content/drive/MyDrive/project/{EXPERIMENT_NAME}_finetuned"

print(f"📂 Experiment: {EXPERIMENT_NAME}")
print(f"📁 Training results will be saved to: {OUTPUT_DIR}")
print(f"📁 Final LoRA model will be saved to: {SAVE_DIR}")

BATCH_SIZE = 4
NUM_EPOCHS = 3
LEARNING_RATE = 1e-3
TARGET_SR = 16000

device = "cuda" if torch.cuda.is_available() else "cpu"

# Print device here
print("🔍 CUDA available:", torch.cuda.is_available())
if torch.cuda.is_available():
    print("🧠 GPU device:", torch.cuda.get_device_name(0))
    print("🔥 Current device index:", torch.cuda.current_device())
else:
    print("⚠️ Using CPU — training will be very slow")


# ============================================================
# 2. Dataset Definition
# ============================================================
class AudioPromptDataset(Dataset):
    def __init__(self, metadata_path, audio_dir, processor, target_sr=16000):
        with open(metadata_path, "r") as f:
            data = json.load(f)
        # handle nested list like [[{...}]]
        self.metadata = data[0] if isinstance(data, list) and isinstance(data[0], list) else data
        self.audio_dir = Path(audio_dir)
        self.processor = processor
        self.target_sr = target_sr

    def __len__(self):
        return len(self.metadata)

    def __getitem__(self, idx):
        entry = self.metadata[idx]
        signal_id = entry["signal"]
        prompt = entry.get("prompt") or entry.get("original_prompt", "")

        audio_path = self.audio_dir / f"{signal_id}.flac"
        waveform, sr = torchaudio.load(audio_path)

        # convert to mono if stereo
        if waveform.shape[0] > 1:
            waveform = waveform.mean(dim=0, keepdim=True)

        # resample to target_sr
        if sr != self.target_sr:
            waveform = torchaudio.functional.resample(waveform, sr, self.target_sr)

        # extract input features
        input_features = self.processor.feature_extractor(
            waveform.squeeze().numpy(), sampling_rate=self.target_sr
        ).input_features[0]

        # tokenize the text
        labels = self.processor.tokenizer(prompt, return_tensors="pt").input_ids.squeeze(0)

        return {"input_features": input_features, "labels": labels}


# ============================================================
# 3. Collator Function
# ============================================================
@dataclass
class DataCollatorSpeechSeq2SeqWithPadding:
    processor: Any

    def __call__(self, features):
        input_features = [{"input_features": f["input_features"]} for f in features]
        batch = self.processor.feature_extractor.pad(input_features, return_tensors="pt")

        label_features = [{"input_ids": f["labels"]} for f in features]
        labels_batch = self.processor.tokenizer.pad(label_features, return_tensors="pt")

        labels = labels_batch["input_ids"].masked_fill(labels_batch.attention_mask.ne(1), -100)

        # remove duplicate BOS if exists
        if (labels[:, 0] == self.processor.tokenizer.bos_token_id).all().cpu().item():
            labels = labels[:, 1:]

        batch["labels"] = labels
        return batch


# ============================================================
# 4. Load Processor and Dataset
# ============================================================
feature_extractor = WhisperFeatureExtractor.from_pretrained(MODEL_NAME)
tokenizer = WhisperTokenizer.from_pretrained(MODEL_NAME, language=LANGUAGE, task=TASK)
processor = WhisperProcessor.from_pretrained(MODEL_NAME, language=LANGUAGE, task=TASK)

train_dataset = AudioPromptDataset(TRAIN_METADATA_PATH, TRAIN_AUDIO_DIR, processor, TARGET_SR)
valid_dataset = (
    AudioPromptDataset(VALID_METADATA_PATH, VALID_AUDIO_DIR, processor, TARGET_SR)
    if VALID_METADATA_PATH.exists()
    else None
)
collator = DataCollatorSpeechSeq2SeqWithPadding(processor)

# ============================================================
# 4.5. Quick Subset Training Mode (for fast loss verification)
# ============================================================

# ✅ Toggle mode: True = small subset for quick validation / False = full dataset for final training
USE_SMALL_SUBSET = False    # ⬅️ Change to False for full training
SUBSET_SIZE = 500           # Number of samples to use in subset
EPOCHS_SMALL = 1            # Use fewer epochs for quick test

if USE_SMALL_SUBSET:
    from torch.utils.data import Subset
    import random

    random.seed(42)
    indices = random.sample(range(len(train_dataset)), min(SUBSET_SIZE, len(train_dataset)))
    train_dataset = Subset(train_dataset, indices)

    print(f"🧪 Using a small subset of {len(indices)} samples for quick loss check.")
    NUM_EPOCHS = EPOCHS_SMALL  # Temporarily set fewer epochs for subset training
else:
    print("🏋️‍♀️ Using the full training dataset for final training.")


# ============================================================
# 5. Load Whisper Model (16-bit) and Prepare for LoRA
# ============================================================
model = WhisperForConditionalGeneration.from_pretrained(
    MODEL_NAME,
    # torch_dtype=torch.float8_e4m3fn if torch.cuda.is_available() else torch.float8_e4m3fnuz,
    dtype=torch.float16,
    device_map="auto"
)

model.config.forced_decoder_ids = None
model.config.suppress_tokens = []

model = prepare_model_for_kbit_training(model)

# LoRA setup
# lora_config = LoraConfig(
#     r=16,
#     lora_alpha=32,
#     target_modules=["q_proj", "v_proj"],
#     lora_dropout=0.05,
#     bias="none",
#     task_type="SEQ_2_SEQ_LM",
# )


lora_config = LoraConfig(
    r=32,
    lora_alpha=64,
    target_modules = ["q_proj", "v_proj", "fc1", "fc2"],
    lora_dropout=0.1,
    bias="lora_only",
    task_type="SEQ_2_SEQ_LM",
)
model = get_peft_model(model, lora_config)
print("Trainable parameters:")
model.print_trainable_parameters()
# ============================================================
# 6. Evaluation Metric (WER)
# ============================================================
wer_metric = evaluate.load("wer")

def compute_metrics(pred):
    pred_ids = pred.predictions
    label_ids = pred.label_ids
    label_ids[label_ids == -100] = tokenizer.pad_token_id

    pred_str = tokenizer.batch_decode(pred_ids, skip_special_tokens=True)
    label_str = tokenizer.batch_decode(label_ids, skip_special_tokens=True)

    wer = 100 * wer_metric.compute(predictions=pred_str, references=label_str)
    return {"wer": wer}


# ============================================================
# 7. Trainer Setup
# ============================================================
training_args = Seq2SeqTrainingArguments(
    #output_dir="./whisper_lora_results",
    output_dir=OUTPUT_DIR,
    per_device_train_batch_size=BATCH_SIZE,
    gradient_accumulation_steps=1,
    learning_rate=LEARNING_RATE,
    warmup_steps=50,
    num_train_epochs=NUM_EPOCHS,
    eval_strategy="no",
    # eval_strategy="epoch" if valid_dataset else "no",
    save_strategy="epoch",
    logging_strategy="steps",
    logging_steps=1,
    logging_dir="./logs",
    fp16=True,
    remove_unused_columns=False,
    label_names=["labels"],
    report_to="none",
    per_device_eval_batch_size=4,
    # eval_accumulation_steps=1,
    # predict_with_generate=False
)

trainer = Seq2SeqTrainer(
    args=training_args,
    model=model,
    train_dataset=train_dataset,
    eval_dataset=valid_dataset if valid_dataset else None,
    data_collator=collator,
    tokenizer=processor,
    compute_metrics=compute_metrics if valid_dataset else None,
)

model.config.use_cache = False  # disable for training

# ============================================================
# 8. Train and Save
# ============================================================
from types import MethodType

def safe_forward(self, *args, **kwargs):
    # Drop 'input_ids' if it appears (Trainer sometimes injects it)
    kwargs.pop("input_ids", None)
    return self.base_model.forward(*args, **kwargs)

# Attach to PEFT model
model.forward = MethodType(safe_forward, model)

trainer.train()

#SAVE_DIR = "./whisper_lora_finetuned"
os.makedirs(SAVE_DIR, exist_ok=True)
model.save_pretrained(SAVE_DIR)
processor.save_pretrained(SAVE_DIR)
print(f"✅ Training complete! Model saved to: {SAVE_DIR}")

📂 Experiment: whisper_lora_exp4
📁 Training results will be saved to: /content/drive/MyDrive/project/whisper_lora_exp4_results
📁 Final LoRA model will be saved to: /content/drive/MyDrive/project/whisper_lora_exp4_finetuned
🔍 CUDA available: True
🧠 GPU device: Tesla T4
🔥 Current device index: 0
🏋️‍♀️ Using the full training dataset for final training.
Trainable parameters:
trainable params: 9,584,640 || all params: 251,172,096 || trainable%: 3.8160


/tmp/ipython-input-1610030668.py:250: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = Seq2SeqTrainer(
The model is already on multiple devices. Skipping the move to device specified in `args`.
/usr/local/lib/python3.12/dist-packages/torchaudio/_backend/utils.py:213: UserWarning: In 2.9, this function's implementation will be changed to use torchaudio.load_with_torchcodec` under the hood. Some parameters like ``normalize``, ``format``, ``buffer_size``, and ``backend`` will be ignored. We recommend that you port your code to rely directly on TorchCodec's decoder instead: https://docs.pytorch.org/torchcodec/stable/generated/torchcodec.decoders.AudioDecoder.html#torchcodec.decoders.AudioDecoder.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/torchaudio/_backend/ffmpeg.py:88: UserWarning: torio.io._streaming_media_decoder.StreamingMediaDecoder has been deprecated. This deprecat

Step,Training Loss
1,8.002300
2,8.377600
3,8.242400
4,8.580600
5,6.819000
6,8.064700
7,6.524000
8,6.169600
9,5.515800
10,5.230200


/usr/local/lib/python3.12/dist-packages/torchaudio/_backend/utils.py:213: UserWarning: In 2.9, this function's implementation will be changed to use torchaudio.load_with_torchcodec` under the hood. Some parameters like ``normalize``, ``format``, ``buffer_size``, and ``backend`` will be ignored. We recommend that you port your code to rely directly on TorchCodec's decoder instead: https://docs.pytorch.org/torchcodec/stable/generated/torchcodec.decoders.AudioDecoder.html#torchcodec.decoders.AudioDecoder.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/torchaudio/_backend/ffmpeg.py:88: UserWarning: torio.io._streaming_media_decoder.StreamingMediaDecoder has been deprecated. This deprecation is part of a large refactoring effort to transition TorchAudio into a maintenance phase. The decoding and encoding capabilities of PyTorch for both audio and video are being consolidated into TorchCodec. Please see https://github.com/pytorch/audio/issues/3902 for more information. It will be r

✅ Training complete! Model saved to: /content/drive/MyDrive/project/whisper_lora_exp4_finetuned


In [ ]:
!ls /content/drive/MyDrive/project/whisper_lora_finetuned

# Inference on the Fine-Tuned LoRA Model

In [ ]:
from transformers import WhisperProcessor, WhisperForConditionalGeneration
from peft import PeftModel
import numpy as np
import torch
import torchaudio
from pathlib import Path
import os
import json



class AudioPromptDataset(torch.utils.data.Dataset):
    def __init__(self, metadata_path, audio_dir, processor, target_sr=16000):
        with open(metadata_path, "r") as f:
            data = json.load(f)
        # handle nested list like [[{...}]]
        self.metadata = data[0] if isinstance(data, list) and isinstance(data[0], list) else data
        self.audio_dir = Path(audio_dir)
        self.processor = processor
        self.target_sr = target_sr

    def __len__(self):
        return len(self.metadata)

    def __getitem__(self, idx):
        entry = self.metadata[idx]
        signal_id = entry["signal"]
        prompt = entry.get("prompt") or entry.get("original_prompt", "")

        audio_path = self.audio_dir / f"{signal_id}.flac"
        waveform, sr = torchaudio.load(audio_path)

        # convert to mono if stereo
        if waveform.shape[0] > 1:
            waveform = waveform.mean(dim=0, keepdim=True)

        # resample
        if sr != self.target_sr:
            waveform = torchaudio.functional.resample(waveform, sr, self.target_sr)

        # extract input features
        input_features = self.processor.feature_extractor(
            waveform.squeeze().numpy(), sampling_rate=self.target_sr
        ).input_features[0]

        # tokenize text
        labels = self.processor.tokenizer(prompt, return_tensors="pt").input_ids.squeeze(0)

        return {"input_features": input_features, "labels": labels, "metadata": entry}

DATA_ROOT = Path("/content/drive/MyDrive")
TRAIN_AUDIO_DIR = DATA_ROOT / "train" / "signals"
VALID_AUDIO_DIR = DATA_ROOT / "valid" / "signals"
TRAIN_METADATA_PATH = DATA_ROOT / "metadata" / "train_metadata.json"
VALID_METADATA_PATH = DATA_ROOT / "metadata" / "valid_metadata.json"  # optional

MODEL_NAME = "openai/whisper-small"  # or "openai/whisper-large-v2"
LANGUAGE = "English"
TASK = "transcribe"

EXPERIMENT_ID = "exp4"
EXPERIMENT_NAME = f"whisper_lora_{EXPERIMENT_ID}"

OUTPUT_DIR = f"/content/drive/MyDrive/project/{EXPERIMENT_NAME}_results"
SAVE_DIR = f"/content/drive/MyDrive/project/{EXPERIMENT_NAME}_finetuned"

# ===== Re-initialize core settings =====
DATA_ROOT = Path("/content/drive/MyDrive")

MODEL_NAME = "openai/whisper-small"
EXPERIMENT_ID = "exp4"
EXPERIMENT_NAME = f"whisper_lora_{EXPERIMENT_ID}"

OUTPUT_DIR = f"/content/drive/MyDrive/project/{EXPERIMENT_NAME}_results"
SAVE_DIR = f"/content/drive/MyDrive/project/{EXPERIMENT_NAME}_finetuned"

TARGET_SR = 16000
LANGUAGE = "English"
TASK = "transcribe"
device = "cuda" if torch.cuda.is_available() else "cpu"

# ===== Load processor (needed for feature extraction & decoding) =====
processor = WhisperProcessor.from_pretrained(SAVE_DIR)
print("✅ Processor loaded successfully!")


TEST_AUDIO_DIR = DATA_ROOT / "valid" / "signals"
TEST_METADATA_PATH = DATA_ROOT / "metadata" / "valid_metadata.json"

test_dataset = AudioPromptDataset(TEST_METADATA_PATH, TEST_AUDIO_DIR, processor, TARGET_SR)

# --- load saved adapter on top of base ---
SAVE_DIR = "/content/drive/MyDrive/project/whisper_lora_finetuned"
base_model = WhisperForConditionalGeneration.from_pretrained(MODEL_NAME)
model = PeftModel.from_pretrained(base_model, SAVE_DIR)  # LoRA adapters
processor = WhisperProcessor.from_pretrained(SAVE_DIR)

model.to(device)
model.eval()

# (Optional) If you want language/task tokens automatically:
# ids = processor.get_decoder_prompt_ids(language=LANGUAGE.lower(), task=TASK)
# model.generation_config.forced_decoder_ids = ids  # or leave None if you trained that way

def transcribe_sample(audio_path, max_new_tokens=128):
    waveform, sr = torchaudio.load(audio_path)

    # mono
    if waveform.shape[0] > 1:
        waveform = waveform.mean(dim=0, keepdim=True)

    # resample
    if sr != TARGET_SR:
        waveform = torchaudio.functional.resample(waveform, sr, TARGET_SR)

    # extract features -> numpy array [80, T]
    feats = processor.feature_extractor(
        waveform.squeeze().numpy(), sampling_rate=TARGET_SR
    ).input_features[0]  # np.ndarray

    # convert to torch tensor with batch dim and move to device
    input_features = torch.from_numpy(np.array(feats)).unsqueeze(0).to(device)

    # fp16 autocast helps on CUDA when model is half / mixed-precision
    with torch.no_grad():
        # If you trained with fp16, you can also do:
        # with torch.autocast(device_type="cuda", dtype=torch.float16, enabled=(device=="cuda")):
        generated_ids = model.generate(
            input_features=input_features,     # <-- KEY FIX: use keyword input_features
            max_new_tokens=max_new_tokens
        )

    transcription = processor.batch_decode(generated_ids, skip_special_tokens=True)[0]
    return transcription

for i in range(min(200, len(test_dataset))):
    print(f'i={i}')
    sid = test_dataset.metadata[i]["signal"]
    audio_path = TEST_AUDIO_DIR / f"{sid}.flac"
    pred_text = transcribe_sample(audio_path)
    ref_text  = test_dataset.metadata[i].get("prompt", "")
    loss_level  = test_dataset.metadata[i].get("hearing_loss", "")
    print(f"🎵 {sid}")
    print(f"Loss Level: {loss_level}")
    print(f"Prediction: {pred_text}")
    print(f"Reference : {ref_text}\n")


✅ Processor loaded successfully!


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/967M [00:00<?, ?B/s]

generation_config.json: 0.00B [00:00, ?B/s]

i=0


/usr/local/lib/python3.12/dist-packages/torchaudio/_backend/utils.py:213: UserWarning: In 2.9, this function's implementation will be changed to use torchaudio.load_with_torchcodec` under the hood. Some parameters like ``normalize``, ``format``, ``buffer_size``, and ``backend`` will be ignored. We recommend that you port your code to rely directly on TorchCodec's decoder instead: https://docs.pytorch.org/torchcodec/stable/generated/torchcodec.decoders.AudioDecoder.html#torchcodec.decoders.AudioDecoder.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/torchaudio/_backend/ffmpeg.py:88: UserWarning: torio.io._streaming_media_decoder.StreamingMediaDecoder has been deprecated. This deprecation is part of a large refactoring effort to transition TorchAudio into a maintenance phase. The decoding and encoding capabilities of PyTorch for both audio and video are being consolidated into TorchCodec. Please see https://github.com/pytorch/audio/issues/3902 for more information. It will be r

🎵 c657dc0bd1f822f8ff7104a1
Loss Level: Moderate
Prediction: true love is known i am true to you more
Reference : do you know i am truly alone

i=1
🎵 5ab5a95be3c18dc173fdc18e
Loss Level: Moderate
Prediction: down a hands in hands we are on the floor
Reference : do not ask me that we are partners

i=2
🎵 795996ed006725199e5efc0b
Loss Level: Moderate
Prediction: i have to go be on me for all may
Reference : i had to walk before i made

i=3
🎵 ff6946553173184eae036062
Loss Level: Mild
Prediction: n-apteoane is far from me
Reference : that is the way it is going to be

i=4
🎵 95e1416bf854594d6716ab25
Loss Level: Moderate
Prediction: but we have to stand again
Reference : what we have is sunday again

i=5
🎵 29dbd8d2300642cfd31b4604
Loss Level: No Loss
Prediction: but when i come around
Reference : but when i come around

i=6
🎵 1d887341bdf775c49d8a0c30
Loss Level: No Loss
Prediction: you can know my hand
Reference : you can hold my hand

i=7
🎵 77dda6054a517033716f00ff
Loss Level: Mild
Prediction

In [ ]:
!ls -R /content/drive/MyDrive/project/whisper_lora_results | head -n 40



In [ ]:
import json

path = "/content/drive/MyDrive/project/whisper_lora_results/checkpoint-6603/trainer_state.json"
with open(path) as f:
    state = json.load(f)

for log in state["log_history"][-10:]:
    print(log)



In [ ]:
!nvidia-smi



Thu Nov  6 12:20:11 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   46C    P0             26W /   70W |    1304MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

# Calculate Word Error Rate (WER)

In [ ]:
import time
import evaluate

# Load WER metric
wer_metric = evaluate.load("wer")
print("✅ Loaded WER metric successfully!")

predictions, references = [], []

for i in range(len(test_dataset)):
    entry = test_dataset.metadata[i]
    audio_path = TEST_AUDIO_DIR / f"{entry['signal']}.flac"

    start = time.time()
    print(f"\n🟢 [{i+1}/{len(test_dataset)}] Processing {audio_path.name} ...", flush=True)

    pred = transcribe_sample(audio_path)
    duration = time.time() - start
    print(f"✅ Done in {duration:.2f} sec | Pred: {pred[:50]}...", flush=True)

    predictions.append(pred)
    references.append(entry.get("prompt", ""))

wer = wer_metric.compute(predictions=predictions, references=references)
print(f"\n WER on valid set: {wer * 100:.2f}%")



✅ Loaded WER metric successfully!

🟢 [1/1175] Processing c657dc0bd1f822f8ff7104a1.flac ...
✅ Done in 0.57 sec | Pred: true love is known i am true to you more...

🟢 [2/1175] Processing 5ab5a95be3c18dc173fdc18e.flac ...
✅ Done in 0.48 sec | Pred: down a hands in hands we are on the floor...

🟢 [3/1175] Processing 795996ed006725199e5efc0b.flac ...
✅ Done in 0.45 sec | Pred: i have to go be on me for all may...

🟢 [4/1175] Processing ff6946553173184eae036062.flac ...
✅ Done in 0.45 sec | Pred: n-apteoane is far from me...

🟢 [5/1175] Processing 95e1416bf854594d6716ab25.flac ...
✅ Done in 0.37 sec | Pred: but we have to stand again...

🟢 [6/1175] Processing 29dbd8d2300642cfd31b4604.flac ...
✅ Done in 0.36 sec | Pred: but when i come around...

🟢 [7/1175] Processing 1d887341bdf775c49d8a0c30.flac ...
✅ Done in 0.65 sec | Pred: you can know my hand...

🟢 [8/1175] Processing 77dda6054a517033716f00ff.flac ...
✅ Done in 0.44 sec | Pred: we can back it we can be seven...

🟢 [9/1175] Processing 8e